In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
from abc import ABCMeta, abstractmethod
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, 'C:/Users/peter/Desktop/volatility-forecasting/Model')
from weights import WeightMethod

## Volatilitás modellezése MIDAS-sal

Az elmúlt alkalommal beszéltek alapján, elkezdtem a MIDAS-nak az OOP megvalósítását.<br>A napi volatilitás becsléséhez, csak napi frekvenciájú adataink állnak rendelkezésre, így csak ha a késleltetéseket és napi frekvenciájú változókat használok fel mint magyarázóváltozók akkor a modell leírása a következő:

$$\sigma_t^2  = \alpha_0 + \alpha_1 \sum_{i=1}^{lag} b(1, \theta_1)\sigma_{t-i}^2 + \sum_{i=1}^{N} \beta_i \sum_{j=1}^{lag} b(1, \theta) X_{t-j}^{(i)} $$

ahol<br>
$r_t = log(P_t) - log(P_{t-1})$<br>
$ \sigma_t^2 = r_t^2$<br> 
$b(1, \theta_1)$ a Béta súlyfüggvény, amely első paramétere 1-ben le van rögzítve<br>
$X^{(i)}$ az i-edik magyarázóváltozó

A GARCH-MIDAS-os papírok alapján, ha még alacsonyabb frekvenciájú, mondjuk havi változókat szeretnénk beleépíteni a modellbe, akkor azokat a hónapon keresztül konstansnak vesszük. Ezen változóknak szintén vehetjük késleltettjeit is, és azokat Béta súlyfüggvénnyel számolhatjuk. Ha szeretnénk a napi volatilitásokat becsülni napi és havi adatainkkal akkor a modell leírása a következő:

$$\sigma_{M, t}^2  = \alpha_0 + \alpha_1 \sum_{i=1}^{lag} b(1, \theta)\sigma_{M, t-i}^2 + \sum_{i=1}^{N} \beta_i \sum_{j=1}^{lag} b(1, \theta) X_{M, t-j}^{(i)} + \sum_{i=1}^{N} {\gamma_i \sum_{j=1}^{lag} {b(1, \theta)X_{M-j}^{(i)}}}$$

In [9]:
class MIDAS(object):
    def __init__(self, lag = 22, init_params = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], *args):
        self.lag = lag
        self.init_params = init_params
        self.args = args
        
    def create_lagged_variable(self, data, name):
        """
        Az eredményváltozóm, jelen esetben a volatilitás késletetéseit elkészítő függvény.

        Parameters
        ----------
        data : DataFrame
            Ami tartalmazza a 'Volatility' oszlopot.

        Returns
        -------
        new_df : DataFrame
            DESCRIPTION.

        """
        ## Létrehozok egy új dataframe-et, amibe fogom behelyzeni a késleltett értékeit az eredményváltozómnak.
        new_df = pd.DataFrame(data = {name: data})
        for i in range(self.lag):
            ## Annyi késleltetést készítek, amennyit a self.lag-ban megadtam
            new_df['Lag {number}'.format(number = i + 1)] = new_df[name].shift(i + 1).fillna(0)
        return new_df

    def model(self, params, x):
        model = params[0] 
        model += params[1] * WeightMethod().x_weighted_beta(x[:, :self.lag], [1.0, params[2]])
        model += params[3] * WeightMethod().x_weighted_beta(x[:, self.lag:self.lag*2], [1.0, params[4]])
        model += params[5] * WeightMethod().x_weighted_beta(x[:, self.lag*2:], [1.0, params[6]])
        return model
    
    def loglikelihood(self, params, y, x):
        return np.sum((y - self.model(params, x)) ** 2)
    
    def fit(self, data):
        data1_matrix = self.create_lagged_variable(data.iloc[:, 0], data.columns[0])
        X1 = data1_matrix.iloc[:, -self.lag:].values
        data2_matrix = self.create_lagged_variable(data.iloc[:, 1], data.columns[1])
        X2 = data2_matrix.iloc[:, -self.lag:].values
        data3_matrix = self.create_lagged_variable(data.iloc[:, 2], data.columns[2])
        X3 = data3_matrix.iloc[:, -self.lag:].values

        
        self.X = np.hstack((X1, X2, X3))
        self.y = data1_matrix.iloc[:, :-self.lag].values
        
        
        res = minimize(self.loglikelihood, 
                       self.init_params,
                       args = (self.y, self.X),
                       method = 'l-bfgs-b',
                       options = {'disp': False})
        self.opt = res
        return print(res)

In [10]:
df = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/data/Stocks/AMD.csv')
df['Unnamed: 0'] = df['Unnamed: 0'].astype("datetime64[ms]")
df = df.rename(columns = {"Unnamed: 0": 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
df['LogReturn'] = np.log(df.Close).diff().fillna(0)*100
df['Volatility'] = df['LogReturn'] ** 2.0
df['Volatility'] = df['Volatility'].rolling(22).mean().fillna(0)
df = df.iloc[1022:2022, :]
df.set_index(df.Date, inplace = True)

vix = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/data/Macroeconomic/VIXCLS.csv')
vix = vix[vix.VIXCLS != '.'].reset_index(drop=True)
vix['DATE'] = vix['DATE'].astype("datetime64[ms]")
vix = vix.rename(columns = {'DATE': 'Date', 'VIXCLS': 'VIX'})
vix = vix[(vix.Date >= df.Date.min()) & (vix.Date <= df.Date.max())]
vix['VIX'] = vix['VIX'].astype('float64')
vix.set_index(vix.Date, inplace = True)

wti = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/data/Macroeconomic/DCOILWTICO.csv')
wti = wti[wti.Value != '.'].reset_index(drop=True)
wti['Date'] = wti['Date'].astype("datetime64[ms]")
wti = wti.rename(columns = {'Date': 'Date', 'Value': 'WTI'})
wti = wti[(wti.Date >= df.Date.min()) & (wti.Date <= df.Date.max())]
wti['WTI'] = wti['WTI'].astype('float64')
wti.set_index(wti.Date, inplace = True)


data = pd.concat([df.Volatility, vix.VIX], axis = 1)
data = pd.concat([data, wti.WTI], axis = 1)
data = data.fillna(method = 'bfill')

In [11]:
model = MIDAS(lag = 22)
model.fit(data)

      fun: 47993066.386290364
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([  0.        , -36.50784492,   0.        ,   0.        ,
         0.        ,   3.7252903 ,   0.        ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 416
      nit: 28
     njev: 52
   status: 0
  success: True
        x: array([ 8.56745681e+00, -5.11283584e-08,  3.70478515e+00,  9.41648187e-07,
        4.53511681e+00,  2.01412988e-07,  1.07306181e+01])
